In [1]:
import numpy as np
import gc
import pandas as pd
import torch
from torch import nn
from torch_geometric.nn import GCNConv, SAGEConv, GAE
from torch_geometric.data import Data
from tqdm import tqdm
import polars as pl
from collections import defaultdict
from sklearn.preprocessing import StandardScaler
import os
import sys
sys.path.append('../models')
from utils import *
from models import *


device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

c:\Users\wikid\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cpu


In [4]:
# use features extracted by Node2Vec technique that uses Biase Random Walk to extract node embeddings
node2vec_embeddings  = np.load("../output/node2vec_embeddings.npy")